# Gemma cup detection

## Imports

In [ ]:
import os
import glob
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead, RegionProposalNetwork

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from engine import train_one_epoch, evaluate
import transforms as T

from skimage import io, transform
from skimage.color import rgb2gray

## Define constants

In [ ]:
data_path = os.path.join("..", "data_in")
images_path =os.path.join(data_path, "images")

## Dataset

### Albumentations

In [ ]:
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

### Create class

In [1]:
class GemmaDataset(Dataset):
    def __init__(self, csv, transform=None):
        self.boxes = csv.copy()
        self.images = list(self.boxes.filename.unique())
        self.transforms = transform

    def __len__(self):
        return self.boxes.shape[0]
    
    def load_boxes(self, idx):
        boxes = self.boxes[self.boxes.filename == self.images[idx]].dropna()
        size = boxes.shape[0]
        if size > 0:
            boxes = boxes[['x', 'y', 'width', 'height']].values
            boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
            boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
            return size, boxes
        return 0, []
    
    def load_image(self, idx):
        return rgb2gray(
            io.imread(
                os.path.join(
                    images_path, 
                    self.images[idx]
                )
            )
        )
    
    def __getitem__(self, index):
        num_box, boxes = self.load_boxes(index) # return list of [xmin, ymin, xmax, ymax]
        img = self.load_image(index) # return an image

        if num_box > 0:
          boxes = torch.as_tensor(boxes, dtype=torch.float32)
        else:
          #negative example, ref: https://github.com/pytorch/vision/issues/2144
          boxes = torch.zeros((0, 4), dtype=torch.float32)

        image_id = torch.tensor([index])
        target = {
            "boxes":boxes,
            "labels":torch.ones((num_box,), dtype=torch.int64),
            "image_id": image_id,
            "area": torch.as_tensor(
                (boxes[:, 3] - boxes[:, 1])*(boxes[:, 2] - boxes[:, 0]), 
                dtype=torch.float32
            ),
            "iscrowd": torch.zeros((num_box,), dtype=torch.int64),
        }

        if self.transforms is not None:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
        
        return img, target, image_id        

NameError: name 'Dataset' is not defined

### Test dataset

In [ ]:
ds = GemmaDataset(pd.read_csv(os.path.join(data_path, "boxes.csv")))
len(ds)

In [ ]:
test_index = 1

#### Test images

In [ ]:
ds.images[:5]

#### Test dataframe

In [ ]:
ds.boxes.sort_values(["filename"]).head()

In [ ]:
ds.boxes[ds.boxes.filename == ds.images[test_index]]

#### Test boxes

In [ ]:
len(ds.load_boxes(0))

In [ ]:
ds.load_boxes(test_index)

In [ ]:
ds.load_boxes(test_index)[0:1]

#### Test load images

In [ ]:
img = ds.load_image(test_index)
io.imshow(img) 
io.show()

#### Test item selection

In [ ]:
ds[test_index][1]

## RPN

In [ ]:
fasterRCNN = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# Define RPN 
anchor_generator = AnchorGenerator(
    sizes=tuple([(16, 32, 64, 128, 256) for _ in range(5)]), # let num of tuple equal to num of feature maps
    aspect_ratios=tuple([(0.75, 0.5, 1.25) for _ in range(5)]),  # ref: https://github.com/pytorch/vision/issues/978
)

rpn_head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])

fasterRCNN.rpn = RegionProposalNetwork(
    anchor_generator= anchor_generator, head= rpn_head,
    fg_iou_thresh= 0.7, bg_iou_thresh=0.3,
    batch_size_per_image=48, # use fewer proposals
    positive_fraction = 0.5,
    pre_nms_top_n=dict(training=200, testing=100),
    post_nms_top_n=dict(training=160, testing=80),
    nms_thresh = 0.7
)

## Fast R-CNN

In [ ]:
in_features = fasterRCNN.roi_heads.box_predictor.cls_score.in_features #get number of features
fasterRCNN.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes = 2)
fasterRCNN.roi_heads.fg_bg_sampler.batch_size_per_image = 24
fasterRCNN.roi_heads.fg_bg_sampler.positive_fraction = 0.5

## Train

### Create data loaders

In [ ]:
sizes: tuple = (0.8, 0.20)
batch_size = 10

df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "boxes.csv"))

dataset_size = len(df)
indices = [i for i in df.index]

split_train = int(np.floor(sizes[0] * dataset_size))
split_test = int(np.floor(sizes[1] * dataset_size)) + split_train

np.random.shuffle(indices)
train_indices, test_indices = (
    indices[:split_train],
    indices[split_train:split_test],
)

df_train = df.iloc[train_indices]
df_test = df.iloc[test_indices]

train_loader = torch.utils.data.DataLoader(
    GemmaDataset(csv=df_train, transform=get_transform(train=True)),
    batch_size=batch_size,
)
test_loader = torch.utils.data.DataLoader(
    GemmaDataset(csv=df_test, transform=get_transform(train=False)),
    batch_size=batch_size,
)

### Train loop

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# move model to the right device
fasterRCNN.to(device)

params = [p for p in fasterRCNN.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(
    params, 
    lr=0.0005, 
    betas=(0.9, 0.999), 
    weight_decay=0.0005
)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=3, 
    gamma=0.1
)
metric_collector = []
num_epochs = 15
for epoch in range(num_epochs):
    # train for one epoch, printing every 5 iterations
    metric_logger = train_one_epoch(
        fasterRCNN, 
        optimizer, 
        train_loader, 
        device, 
        epoch, 
        print_freq=5
    )
    metric_collector.append(metric_logger)
    # update the learning rate
    lr_scheduler.step()
    # Evaluate with validation dataset
    metric_logger_val = validate(fasterRCNN, val_data_loader, device, print_freq=5)
    #save checlpoint
    torch.save(
        fasterRCNN.state_dict(), 
        os.path.join( weights_path,'fasterRCNN_ep'+str(epoch)+'.pth')
    )
